# MCNN RSNA

## Settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/Colab_Notebooks/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download deltaechov/rnsamamographt512px8bit

In [ ]:
! unzip rnsamamographt512px8bit.zip

In [ ]:
pip install torchmetrics

In [ ]:
import pandas as pd
import os
from skimage import io, transform, color
from numpy.ma.core import MaskedConstant
import torch
import torch.utils.data.dataloader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import time
from torch.utils.data import RandomSampler, DataLoader, random_split, Dataset
import numpy as np
from sklearn import metrics, svm
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, balanced_accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from morph2d import Morph2d
from pseudoinversemod import pseudoInverse
from tabulate import tabulate
import datetime
import random
import scipy
from torchmetrics.functional import roc
from CIDeLong import delong_roc_variance
from scipy import stats
from MCNN import MCNN
from breastcancerdataset import BreastCancerDataset

## Data Preparation

In [ ]:
rnsa = pd.read_csv('/content/train.csv')

# Drop duplicates
cases = rnsa.patient_id.unique()
print("Initial number of patients: ", len(cases))
print("Initial number of images: ", len(rnsa))

cleaned_rnsa = rnsa.drop_duplicates(subset = ["patient_id", "laterality", "view"])
cleaned_rnsa = cleaned_rnsa.drop(index=cleaned_rnsa[cleaned_rnsa["view"]=='AT'].index)
cleaned_rnsa = cleaned_rnsa.drop(index=cleaned_rnsa[cleaned_rnsa["view"]=='LM'].index)
cleaned_rnsa = cleaned_rnsa.drop(index=cleaned_rnsa[cleaned_rnsa["view"]=='ML'].index)
cleaned_rnsa = cleaned_rnsa.drop(index=cleaned_rnsa[cleaned_rnsa["view"]=='LMO'].index)

cleaned_cases = cleaned_rnsa.patient_id.unique()

ordered_index = []
for id in cleaned_cases:
    aux = cleaned_rnsa[cleaned_rnsa["patient_id"]==id]
    ordered_index.append(aux[aux["laterality"]== 'L'][aux["view"]=='CC'].index[0])
    ordered_index.append(aux[aux["laterality"]== 'L'][aux["view"]=='MLO'].index[0])
    ordered_index.append(aux[aux["laterality"]== 'R'][aux["view"]=='CC'].index[0])
    ordered_index.append(aux[aux["laterality"]== 'R'][aux["view"]=='MLO'].index[0])

cleaned_rnsa = cleaned_rnsa.reindex(ordered_index)

print("Final number of patients: ", len(cleaned_cases))
print("Final number of images: ", len(cleaned_rnsa))

cleaned_rnsa.to_csv('/content/cleaned_rnsa.csv')

In [ ]:
## Create auxiliar csv

# Remove deuplicates
for id in cleaned_cases:
    aux = cleaned_rnsa[cleaned_rnsa["patient_id"]==id]
    if any(aux["cancer"]==1):
        aux["cancer"] = 1
        cleaned_rnsa[cleaned_rnsa["patient_id"]==id] = aux

auxiliar_rnsa = cleaned_rnsa.drop_duplicates(subset = ["patient_id"])

# Choose dataset size
positives = auxiliar_rnsa[auxiliar_rnsa["cancer"]==1].iloc[0:500,:]
negatives = auxiliar_rnsa[auxiliar_rnsa["cancer"]==0].iloc[0:500,:]
frames = [positives, negatives]
auxiliar_rnsa = pd.concat(frames)

auxiliar_rnsa.to_csv('/content/auxiliar_rnsa.csv')

In [ ]:
path_original_folder = '/content/train_images_512px8bit/train_images'
path_cases_df = '/content/auxiliar_rnsa.csv'
path_complete_df = '/content/cleaned_rnsa.csv'
path_train_folder = '/content/Train_RNSA'
path_test_folder = '/content/Test_RNSA'
seed = 28

rnsa_df = pd.read_csv(path_cases_df)
rnsa_complete = pd.read_csv(path_complete_df)
rnsa_df = rnsa_df.sample(frac=1, random_state=seed)

train_indices, test_indices = train_test_split(list(range(len(rnsa_df))),
                                               test_size=0.3,
                                               stratify=rnsa_df["cancer"],
                                               random_state = seed)

train_rnsa = rnsa_df.loc[train_indices]
test_rnsa = rnsa_df.loc[test_indices]
train_rnsa.to_csv('/content/train_rnsa.csv')
test_rnsa.to_csv('/content/test_rnsa.csv')
test_rnsa["patient_id"].to_csv('/content/test_patients_list.csv')

# Obtain indices/names
os.mkdir(path_train_folder)
os.mkdir(path_test_folder)

for patient in train_rnsa["patient_id"]:
    folder_name = os.path.join(path_train_folder, str(patient))
    os.mkdir(folder_name)

    per_folder = rnsa_complete[rnsa_complete["patient_id"]==patient]
    folder_original = os.path.join(path_original_folder, str(patient))

    for name in per_folder["image_id"]:
        full_name = str(name)+".png"
        image_name = os.path.join(folder_original, full_name)
        image = io.imread(image_name)
        fname = os.path.join(folder_name, full_name)
        io.imsave(fname, image)

for patient in test_rnsa["patient_id"]:
    folder_name = os.path.join(path_test_folder, str(patient))
    os.mkdir(folder_name)

    per_folder = rnsa_complete[rnsa_complete["patient_id"]==patient]
    folder_original = os.path.join(path_original_folder, str(patient))

    for name in per_folder["image_id"]:
        full_name = str(name)+".png"
        image_name = os.path.join(folder_original, full_name)
        image = io.imread(image_name)
        fname = os.path.join(folder_name, full_name)
        io.imsave(fname, image)

## Main Code

In [ ]:
# General Settings
batch_size = 560
test_batch_size = 300
num_train_samples = 560
num_test_samples = 300
stop_train = num_train_samples/batch_size
stop_test = num_test_samples/test_batch_size
seed = 28
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

# Training settings

breast_train_dataset = BreastCancerDataset(csv_file='/content/train_rnsa.csv',
                              csv_file_full='/content/cleaned_rnsa.csv',
                              root_dir='/content/Train_RNSA/',
                              transform=transforms.Compose([
                                               Rescale(256),
                                               ToTensor()
                                           ]))

train_loader = torch.utils.data.DataLoader(breast_train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           worker_init_fn=seed_worker,
                                           generator=g)

# Testing settings

breast_test_dataset = BreastCancerDataset(csv_file='/content/test_rnsa.csv',
                              csv_file_full='/content/cleaned_rnsa.csv',
                              root_dir='/content/Test_RNSA/',
                              transform=transforms.Compose([
                                               Rescale(256),
                                               ToTensor()
                                           ]))

test_loader = torch.utils.data.DataLoader(breast_test_dataset,
                                          batch_size=test_batch_size,
                                          shuffle=True,
                                          worker_init_fn=seed_worker,
                                          generator=g)

# Three definitions, each per channel, for weights' independance
model_1 = MCNN()
model_2 = MCNN()
model_3 = MCNN()
model_4 = MCNN()

optimizer_1 = pseudoInverse(params=model_1.parameters(), C=1)
optimizer_2 = pseudoInverse(params=model_2.parameters(), C=1e-3)
optimizer_3 = pseudoInverse(params=model_3.parameters(), C=2)
optimizer_4 = pseudoInverse(params=model_4.parameters(), C=1)

def plot_auc(fpr, tpr, auc_result):

    plt.figure(figsize=(12,7))
    plt.plot(fpr, tpr, linewidth=1, color='blue')
    plt.plot(np.linspace(0, max(fpr), len(fpr)),
             np.linspace(0, max(tpr), len(tpr)),
             "--", linewidth=0.8, color="black")
    plt.title(label="ROC-AUC. MCNN",
             fontsize=15,
             fontweight="bold")
    plt.ylabel("True Positive Rate")
    plt.xlabel("False Positive Rate")
    plt.legend(["AUC: " + str(round(auc_result.item(), 3))], loc='right')
    plt.xlim(left=0)
    plt.ylim(bottom=0)
    plt.xlim(right=max(tpr))
    plt.ylim(top=max(fpr))

def final_classifier(out_train_1, out_train_2, out_train_3, out_train_4, out_test_1,
                     out_test_2, out_test_3, out_test_4, target_train, target_test):
    """This function performs a final classification considering the independent
    DPFs obtained from the MCNN method.
    
    inputs:
      out_trains = DPFs obtained per channel from the training set, it contains
                   the probabilities of both classes [# classes, # subjects].
      out_tests = DPFs obtained per channel from the testing set, it contains
                  the probabilities of both classes [# classes, # subjects].
      target_train = Labels corresponding to the subjects in the training set.
      target_test = Labels corresponding to the subjects in the testing set."""

    np.random.seed(seed)
    
    # Initial definitions
    model1 = GaussianNB()
    model2 = svm.SVC(random_state=seed, probability=True)
    model3 = AdaBoostClassifier(random_state=seed)
    model4 = RandomForestClassifier(random_state=seed)
    model = [model1, model2, model3, model4]

    acc_train = []
    acc_test = []
    acc_balanced = []
    ci_bacc = []
    ci_acc = []
    conf_mat = []
    auc = []
    auc_delong = []
    auc_cov = []
    ci_auc = []
    err = []
    fpr_all = []
    tpr_all = []
    p_fpr_all = []
    p_tpr_all = []

    count = 1
    alpha = 0.95
    confidence = 0.95
    z_value = stats.norm.ppf((1 + confidence) / 2.0)

    acc_train.append("Train set accuracy")
    acc_test.append("Test set accuracy")
    acc_balanced.append("Balanced accuracy")
    ci_bacc.append("CI Balanced acc")
    ci_acc.append("CI Acc")
    auc.append("ROC AUC score")
    auc_delong.append("ROC AUC DeLong")
    ci_auc.append("CI AUC")
    auc_cov.append("AUC COV")
    err.append("Error")
    headers = ["Metrics", "Gaussian Naive Bayes", "SVM",
                        "AdaBoost", "Random Forest"]

    # Input data preparation
    out_train_1 = out_train_1.detach().numpy()
    out_train_2 = out_train_2.detach().numpy()
    out_train_3 = out_train_3.detach().numpy()
    out_train_4 = out_train_4.detach().numpy()
    target_train = target_train.detach().numpy()
    target_test = target_test.detach().numpy()

    target_train = target_train[:,0]
    target_test = target_test[:,0]

    joint_train = np.vstack((out_train_1[:,0],
                             out_train_2[:,0],
                             out_train_3[:,0],
                             out_train_4[:,0]))
    joint_train = np.transpose(joint_train)
    joint_train = pd.DataFrame(joint_train)

    out_test_1 = out_test_1.detach().numpy()
    out_test_2 = out_test_2.detach().numpy()
    out_test_3 = out_test_3.detach().numpy()
    out_test_4 = out_test_4.detach().numpy()

    joint_test = np.vstack((out_test_1[:,0],
                            out_test_2[:,0],
                            out_test_3[:,0],
                            out_test_4[:,0]))
    joint_test = np.transpose(joint_test)
    joint_test = pd.DataFrame(joint_test)

    # Training
    for mod in model:
        trained_model = mod.fit(joint_train,target_train)
        trained_model.fit(joint_train, target_train) #Temporal CORER CON COMENTARIO
        acc_train_ind = metrics.accuracy_score(target_train,
                                               trained_model.predict(joint_train))
        acc_train.append(acc_train_ind)

    # Testing
    for mod in model: 
        predicted = mod.predict(joint_test)
        probs = mod.predict_proba(joint_test)
        acc_test_ind = metrics.accuracy_score(target_test, predicted)
        auc_ind = roc_auc_score(target_test, probs[:,1])
        acc_balanced_ind = balanced_accuracy_score(target_test, predicted)
        fpr, tpr, thresholds = roc_curve(target_test, probs[:,1], pos_label=1)
        p_fpr, p_tpr, _ = roc_curve(target_test, predicted, pos_label=1)
        err_ind = 1 - acc_test_ind

        auc_ind_delong, auc_ind_cov = delong_roc_variance(target_test,
                                                          probs[:,1])
        
        # AUC CIs
        auc_std = np.sqrt(auc_ind_cov)
        lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
        ci_auc_ind = stats.norm.ppf(
            lower_upper_q,
            loc=auc_ind_delong,
            scale=auc_std)

        ci_auc_ind[ci_auc_ind > 1] = 1
        ci_auc_ind[0] = round(ci_auc_ind[0], 3)
        ci_auc_ind[1] = round(ci_auc_ind[1], 3)

        # Acc CIs
        ci_length = z_value * np.sqrt((acc_test_ind * (1 - acc_test_ind)) / joint_test.shape[0])
        ci_lower = acc_test_ind - ci_length
        ci_upper = acc_test_ind + ci_length
        ci_acc_ind = (round(ci_lower, 3), round(ci_upper, 3))

        # Balanced acc CIs
        ci_length = z_value * np.sqrt((acc_balanced_ind * (1 - acc_balanced_ind)) / joint_test.shape[0])
        ci_lower = acc_balanced_ind - ci_length
        ci_upper = acc_balanced_ind + ci_length
        ci_bacc_ind = (round(ci_lower, 3), round(ci_upper, 3))

        acc_test.append(round(acc_test_ind, 3))
        ci_acc.append(ci_acc_ind)
        acc_balanced.append(round(acc_balanced_ind, 3))
        ci_bacc.append(ci_bacc_ind)
        auc_delong.append(round(auc_ind_delong, 3))
        auc_cov.append(round(auc_ind_cov, 3))
        ci_auc.append(ci_auc_ind)
        auc.append(round(auc_ind, 3))
        err.append(round(err_ind, 3))
        fpr_all.append(fpr)
        tpr_all.append(tpr)
        p_fpr_all.append(p_fpr)
        p_tpr_all.append(p_tpr)

        conf_mat_ind = confusion_matrix(target_test, predicted)
        disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat_ind,
                                      display_labels=mod.classes_)
        disp.plot(cmap='Blues')
        disp.ax_.set_title(headers[count])
        count += 1
        print("\n")

        plot_auc(fpr, tpr, auc_ind)
        print("\n")

    # Print metrics resulted
    data = [acc_train, acc_test, ci_acc, acc_balanced,
            ci_bacc, auc_delong, ci_auc, auc_cov, err]
    print(tabulate(data, headers))
    print("\n")

    return fpr_all, tpr_all, p_fpr_all, p_tpr_all


def train():
    """This functions performs the training process"""

    print("Ensembles training process starts.........................")
    model_1.train()
    model_2.train()
    model_3.train()
    model_4.train()

    for batch_idx, sample_batched in enumerate(train_loader):
        if batch_idx > stop_train-1:
          break
        
        # Input data and targets preparation
        data = sample_batched['image'].float()
        data_shape = data.shape
        data_1 = torch.reshape(data[:,0,:,:],(data_shape[0],1,
                                              data_shape[2], data_shape[3]))
        data_2 = torch.reshape(data[:,1,:,:],(data_shape[0],1,
                                              data_shape[2], data_shape[3]))
        data_3 = torch.reshape(data[:,2,:,:],(data_shape[0],1,
                                              data_shape[2], data_shape[3]))
        data_4 = torch.reshape(data[:,3,:,:],(data_shape[0],1,
                                              data_shape[2], data_shape[3]))
        target = sample_batched['label_patient'].int()
        target_1 = sample_batched['label_l_cc'].int()
        target_2 = sample_batched['label_l_mlo'].int()
        target_3 = sample_batched['label_r_cc'].int()
        target_4 = sample_batched['label_r_mlo'].int()
        #target = target.reshape([len(target)])
        data_1, data_2, data_3, data_4 = Variable(data_1), Variable(data_2), Variable(data_3), Variable(data_4)
        target = Variable(target)
        target_1, target_2, target_3, target_4 = Variable(target_1), Variable(target_2), Variable(target_3), Variable(target_4)

        # Independent training per channel
        hiddenOut = model_1.forwardToHidden(data_1) # First channel
        optimizer_1.train(inputs=hiddenOut, targets=target_1)

        hiddenOut = model_2.forwardToHidden(data_2) # Second channel
        optimizer_2.train(inputs=hiddenOut, targets=target_2)

        hiddenOut = model_3.forwardToHidden(data_3) # Third channel
        optimizer_3.train(inputs=hiddenOut, targets=target_3)

        hiddenOut = model_3.forwardToHidden(data_4) # Third channel
        optimizer_3.train(inputs=hiddenOut, targets=target_4)
    
    print("Ensembles training process ends...........................")

def train_accuracy():
    """This functions test the network using the training set.
    
       outputs:
          output_1 = PDFs from first channel [# classes, # subjects]
          output_2 = PDFs from second channel [# classes, # subjects]
          output_3 = PDFs from third channel [# classes, # subjects]
          target = labels of each subject"""

    print("Ensembles training testing starts.........................")
    model_1.eval()
    model_2.eval()
    model_3.eval()
    model_4.eval()

    with torch.no_grad():
        for batch_idx, sample_batched in enumerate(train_loader):
            if batch_idx > stop_train-1:
              break

            # Input data and targets preparation
            data = sample_batched['image'].float()
            data_shape = data.shape
            data_1 = torch.reshape(data[:,0,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_2 = torch.reshape(data[:,1,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_3 = torch.reshape(data[:,2,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_4 = torch.reshape(data[:,3,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            target = sample_batched['label_patient'].int()
            target_1 = sample_batched['label_l_cc'].int()
            target_2 = sample_batched['label_l_mlo'].int()
            target_3 = sample_batched['label_r_cc'].int()
            target_4 = sample_batched['label_r_mlo'].int()
            #target = target.reshape([len(target)])
            data_1, data_2, data_3, data_4 = Variable(data_1), Variable(data_2), Variable(data_3), Variable(data_4)
            target = Variable(target)
            target_1, target_2, target_3, target_4 = Variable(target_1), Variable(target_2), Variable(target_3), Variable(target_4)

            output_1 = model_1.forward(data_1) # First channel
            output_2 = model_2.forward(data_2) # Second channel
            output_3 = model_3.forward(data_3) # Third channel
            output_4 = model_4.forward(data_4) # Fourth channel

    print("Ensembles training testing ends...........................")
    return output_1, output_2, output_3, output_4, target, target_1, target_2, target_3, target_4


def test():
    """This functions test the network using the testing set.
    
       outputs:
          output_1 = PDFs from first channel [# classes, # subjects]
          output_2 = PDFs from second channel [# classes, # subjects]
          output_3 = PDFs from third channel [# classes, # subjects]
          target = labels of each subject"""

    print("Ensembles testing starts..................................")
    model_1.eval()
    model_2.eval()
    model_3.eval()
    model_4.eval()

    with torch.no_grad():
        for batch_idx, sample_batched in enumerate(test_loader):
            if batch_idx > stop_test-1:
              break

            # Input data and targets preparation
            data = sample_batched['image'].float()
            data_shape = data.shape
            data_1 = torch.reshape(data[:,0,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_2 = torch.reshape(data[:,1,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_3 = torch.reshape(data[:,2,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_4 = torch.reshape(data[:,3,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            target = sample_batched['label_patient'].int()
            target_1 = sample_batched['label_l_cc'].int()
            target_2 = sample_batched['label_l_mlo'].int()
            target_3 = sample_batched['label_r_cc'].int()
            target_4 = sample_batched['label_r_mlo'].int()
            #target = target.reshape([len(target)])
            data_1, data_2, data_3, data_4 = Variable(data_1), Variable(data_2), Variable(data_3), Variable(data_4)
            target = Variable(target)
            target_1, target_2, target_3, target_4 = Variable(target_1), Variable(target_2), Variable(target_3), Variable(target_4)

            output_1 = model_1.forward(data_1) # First channel
            output_2 = model_2.forward(data_2) # Second channel
            output_3 = model_3.forward(data_3) # Third channel
            output_4 = model_4.forward(data_4) # Fourth channel
        
    return output_1, output_2, output_3, output_4, target, target_1, target_2, target_3, target_4

# Complete MCNN
init=time.time()
train()
train_time = time.time()
out_train_1, out_train_2, out_train_3, out_train_4, target_train, train_target_1, train_target_2, train_target_3, train_target_4 = train_accuracy()

out_test_1, out_test_2, out_test_3, out_test_4, target_test, test_target_1, test_target_2, test_target_3, test_target_4 = test()
test_time = time.time()
FPR, TPR, P_FPR, P_TPR = final_classifier(out_train_1, out_train_2, out_train_3, out_train_4,
                 out_test_1, out_test_2, out_test_3, out_test_4,
                 target_train, target_test)
ending=time.time()

print("Training time: ", str(datetime.timedelta(seconds = train_time - init)), "\n",
      "Testing time: ", str(datetime.timedelta(seconds = test_time - train_time)), "\n",
      "Final process time: ", str(datetime.timedelta(seconds = ending - test_time)), "\n",
      "Complete process time: ", str(datetime.timedelta(seconds = ending - init)))
print("\n")